# Titanic - Machine Learning From Disaster

## Importing necessary libraries

Some of them are probably not going to be used, but it's really convenient to have them already loaded just in case.

In [1]:
# NumPy and Pandas
import numpy as np
import pandas as pd

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Scalers
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import NuSVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Model Helpers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn import feature_selection

# Cross-validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate

## Setting a Random Seed

This ensures the results can be reproduced.

In [2]:
RANDOM_SEED = 13
np.random.seed(RANDOM_SEED)

## Loading the Datasets

In [3]:
train_df = pd.read_csv("train.csv", sep=",")
test_df = pd.read_csv("test.csv", sep=",")
full_df = train_df.append(test_df)
full_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Exploratory Analysis and Data Visualization

In [4]:
ProfileReport(full_df, title="Titanic Profiling Report").to_widgets()

Generate report structure: 100%|██████████| 1/1 [00:07<00:00,  7.21s/it]


## Plot Charts of Pivot Tables

This helps to understand which groups were more succesful at escaping the shipwreck. The function below helps a lot in the visualization of specific groups.

In [5]:
def plotPieCharts(pvt_table, name):
    labels = ["Survived", "Died"]
    domains = []
    xDomainSize = 5 if len(pvt_table) >= 5 else len(pvt_table)
    for i in range(len(pvt_table)):
            domains.append(
                {
                    "x": [
                        (i % 5) / (xDomainSize),
                        (((i + 1) % 5) / xDomainSize) + (0 ** ((i + 1) % 5))
                    ],
                    "y": [
                        1 - (np.ceil((i + 1) / 5) * (1 / (np.ceil(len(pvt_table) / 5)))),
                        1 - (np.floor(i / 5) * (1 / (np.ceil(len(pvt_table) / 5)))),
                    ]
                }
            )

    traces = []
    values = []
    titles = []

    for idx, entry in pvt_table.iterrows():
        values.append([entry[1], 1 - entry[1]])
        titles.append(str(entry[0]))

    for domain, value, title in zip(domains, values, titles):
        trace = go.Pie(
            title=f"{name}: {title}",
            labels=labels,
            values=value,
            domain=domain,
            hoverinfo="label+percent",
            marker=dict(colors=['royalblue', 'red']),
        )
        traces.append(trace)

    layout = go.Layout(
        width=900,
        autosize=False,
        title=name,
    )
    fig = go.Figure(data=traces, layout=layout)
    fig.update_traces(textposition="inside")
    fig.show()

## Feature Selection and Engineering

Some useful information might be hidden within some of the not so useful original features. Some other features might need some manipulation to improve the model's accuracy.

### Pclass

A passenger's class is an important feature when trying to predict if that passenger was able to survive.

In [6]:
survivalPerClass = full_df[["Pclass", "Survived"]].groupby(["Pclass"], as_index=False).mean().sort_values(
    by="Survived", ascending=False
)

plotPieCharts(survivalPerClass, "Passenger Class")

### Sex

The majority of women survived the Titanic. On the other hand, most men didn't have the same luck.

In [7]:
survivalPerSex = full_df[["Sex", "Survived"]].groupby(["Sex"], as_index=False).mean().sort_values(
    by="Survived", ascending=False
)

plotPieCharts(survivalPerSex, "Passenger Sex")

Let's make the "Sex" feature binary. 

male -> 1

female -> 0

In [8]:
full_df.replace({"Sex" : {"male" : 1, "female" : 0}}, inplace=True)

### SibSp + Parch = Relatives

It's possible to obtain the number of family members in the Titanic for every entry by adding "Parch" and "SibSp". That feature is also important. The majority of people with no family aboard died, as well as people traveling with 4 or more family members.

In [9]:
full_df["Relatives"] = full_df.SibSp + full_df.Parch

In [10]:
survivalPerFamily = full_df[["Relatives", "Survived"]].groupby(["Relatives"], as_index=False).mean()

plotPieCharts(survivalPerFamily, "Relatives")

### Age

Age is a very important factor to predict if a person survived the shipwreck. Children were evacuated first, for example. But first it's important to fill the missing values. To do so, I created a "Title" feature, which allowed me to impute ages more precisely. The Passenger Class is also considered to help improve the imputation precision.

In [11]:
# Creating Title feature
full_df["Title"] = full_df["Name"].str.extract(" ([A-Za-z]+)\.")
full_df.Title.replace(["Mlle", "Ms", "Mme"], "Miss", inplace=True)
full_df.Title.replace(["Lady", "Countess", "Dona"], "Mrs", inplace=True)
full_df.Title.replace(["Major", "Jonkheer", "Don", "Col", "Sir", "Capt"], "Mr", inplace=True)

# Imputing Age based on Title and Passenger Class
age_medians = full_df.groupby(["Title", "Pclass"])["Age"]
full_df.Age = age_medians.transform(lambda x: x.fillna(x.median()))

# Dropping Ttitle feature
full_df.drop(columns=["Title"], inplace=True)

# Plotting Age distribution by Survival
data = [full_df.Age[full_df.Survived == 0], full_df.Age[full_df.Survived == 1]]
labels = ["Died", "Survived"]
colors = ["red", "royalblue"]
fig = ff.create_distplot(data, labels, colors=colors, bin_size=5, show_rug=False, show_hist=False)
fig.update_layout(
    title_text="Passenger Age Distribution by Survival",
    xaxis_title="Age"
)
fig.show()

Furthermore, creating bins for Age is likely to improve the model's performance.

In [12]:
full_df["AgeBin"] = pd.qcut(full_df["Age"], 5)

encoder = LabelEncoder()

full_df["AgeBin"] = encoder.fit_transform(full_df["AgeBin"])

full_df.drop(columns=["Age"], inplace=True)

### Survived, Name, Ticket and Fare -> GroupSurvival

Another potentially useful information is whether someone in a passenger travel group survived. A person that belongs to a group that has survivors is probably more likely to have survived too. To define a travel group it's important to look at a passenger's Last Name, Fare and Ticket.

In [14]:
full_df["LastName"] = full_df.Name.str.split().str[0].str.strip(",")

fare_medians = full_df.groupby(["Pclass", "Sex"])["Fare"]
full_df.Fare = fare_medians.transform(lambda x: x.fillna(x.median()))

initialSurvivalValue = 1
full_df["GroupSurvival"] = initialSurvivalValue

familyGroups = full_df[["PassengerId", "Survived", "GroupSurvival", "LastName", "Fare"]].groupby(
    ["LastName", "Fare"]
)

for _, group_df in familyGroups:
    if len(group_df) > 1:
        for idx, entry in group_df.iterrows():
            maxSurvivedStatus = group_df.drop(idx)["Survived"].max()
            minSurvivedStatus = group_df.drop(idx)["Survived"].min()
            passengerId = entry.PassengerId
            if maxSurvivedStatus == 1:
                full_df.loc[full_df["PassengerId"] == passengerId, "GroupSurvival"] = 2
            elif minSurvivedStatus == 0:
                full_df.loc[full_df["PassengerId"] == passengerId, "GroupSurvival"] = 0

In [15]:
travelGroups = full_df[["PassengerId", "Survived", "GroupSurvival", "Ticket"]].groupby(
    ["Ticket"]
)

for _, group_df in travelGroups:
    if len(group_df) > 1:
        for idx, entry in group_df.iterrows():
            if entry.GroupSurvival < 2:
                maxSurvivedStatus = group_df.drop(idx)["Survived"].max()
                minSurvivedStatus = group_df.drop(idx)["Survived"].min()
                passengerId = entry.PassengerId
                if maxSurvivedStatus == 1:
                    full_df.loc[full_df["PassengerId"] == passengerId, "GroupSurvival"] = 2
                elif minSurvivedStatus == 0:
                    full_df.loc[full_df["PassengerId"] == passengerId, "GroupSurvival"] = 0

### Fare

The next feature to analyze is "Fare". The passengers in the first class were more likely to survive the accident. Therefore, it's expected that people that paid a higher price to embark were also more likely live through that event.

In [16]:
data = [full_df.Fare[full_df.Survived == 0], full_df.Fare[full_df.Survived == 1]]
labels = ["Died", "Survived"]
colors = ["red", "royalblue"]
fig = ff.create_distplot(data, labels, colors=colors, bin_size=5, show_rug=False, show_hist=False)
fig.update_layout(
    title_text="Fare Distribution by Survival",
    xaxis_title="Fare"
)
fig.show()

Creating bins for the Fare price is probably a good way to increase algorithm performance. Let's implement those bins:

In [17]:
# Making Bins
full_df["FareBin"] = pd.qcut(full_df["Fare"], 5)

encoder = LabelEncoder()

full_df["FareBin"] = encoder.fit_transform(full_df["FareBin"])

full_df.drop(columns=["Fare"], inplace=True)

## Train/Test Split, Again...

Finally, separate once again the train and test sets and drop the unnecessary features.

In [18]:
train = full_df[:891]
test = full_df[891:]

testIds = test["PassengerId"]

train.drop(
    columns=["PassengerId", "Name", "SibSp", "Parch", "Ticket", "Cabin", "Embarked", "LastName"],
    inplace=True
)
test.drop(
    columns=["PassengerId", "Name", "SibSp", "Parch", "Ticket", "Cabin", "Embarked", "LastName", "Survived"],
    inplace=True
)

print(f"Train set dimensions: {train.shape}")
print(f"Test set dimensions: {test.shape}")

Train set dimensions: (891, 7)
Test set dimensions: (418, 6)


In [19]:
train.head()

,Survived,Pclass,Sex,Relatives,AgeBin,GroupSurvival,FareBin
0,0.0,3,1,1,1,1,0
1,1.0,1,0,1,3,1,4
2,1.0,3,0,0,1,1,1
3,1.0,1,0,1,3,0,4
4,0.0,3,1,0,3,1,1


## Feature Scaling

This technique makes most models perform better. In this case the Standard Scaler works best.

In [20]:
scaler = StandardScaler()
X_train = scaler.fit_transform(train.drop(columns=["Survived"]))
X_train = pd.DataFrame(data=X_train, columns=train.drop(columns=["Survived"]).columns)
X_test = scaler.transform(test)
X_test = pd.DataFrame(data=X_test, columns=test.columns)

y_train = train.Survived

## Iterating Models

It's hard to know which model performs best, thus I decided to try a few of the possible models. Using GridSearchCV also helped to refine hyperparameters.

In [21]:
config = {
    "KNeighborsClassifier": {
        "clf": KNeighborsClassifier(n_neighbors=18, leaf_size=26),
        "parameters": {
            "algorithm": ["auto", "ball_tree"],
            "weights": ["uniform", "distance"],
        }
    },
    "LogisticRegression" : {
        "clf": LogisticRegression(C=0.8, random_state=RANDOM_SEED),
        "parameters": {
            "penalty": ["l1", "l2"],
            "class_weight": [1, 3, 10],
            "dual": [True, False],
            "fit_intercept": [True, False],
            "max_iter": [100, 500, 1000]
        }
    },
    "LinearSVC": {
        "clf": LinearSVC(C=1.6, random_state=RANDOM_SEED),
        "parameters": {
            "penalty": ["l1", "l2"],
            "loss": ["hinge", "squared_hinge"]
        } 
    },
    "RandomForestClassifier": {
        "clf": RandomForestClassifier(n_estimators=100, criterion="gini", random_state=RANDOM_SEED),
        "parameters": {
            "max_depth": [2, 3, 4],
            "min_samples_leaf": [2, 3]
        }
    },
    "GradientBoostingClassifier": {
        "clf": GradientBoostingClassifier(random_state=RANDOM_SEED),
        "parameters": {
            "n_estimators": [10, 50, 100],
            "learning_rate": [0.05, 0.01, 0.1],
            "max_depth": [3, 4, 6],
            "min_samples_leaf": [2, 3]
        }
    },
    "NuSVC": {
        "clf": NuSVC(probability=True, random_state=RANDOM_SEED),
        "parameters": {
            "nu": [0.1, 0.2, 0.5, 0.9, 1, 5]
        }
    },
    "LinearDiscriminantAnalysis": {
        "clf": LinearDiscriminantAnalysis(),
        "parameters": {
            "solver" : ["svd", "lsqr", "eigen"],
        }
    }
}

clfs = config.keys()

In [22]:
results = {"Name": [], "F1 Score": [], "Accuracy": []}
train_pred = {}
test_pred = {}
for name in clfs:
    conf = config[name]
    clf = conf["clf"]
    parameters = conf["parameters"]

    print("=" * 40)
    print("Starting training:", name)
    grid = GridSearchCV(clf, parameters, scoring="roc_auc", cv=10, n_jobs=-1, verbose=2)
    
    print("Number of Features:", X_train.columns.shape[0])
    grid = grid.fit(X_train, y_train)
    best_clf = grid.best_estimator_ 
    
    print("Best classifier:", repr(best_clf))
    model = best_clf.fit(X_train, y_train)
    pred_y = cross_val_predict(best_clf, X_train, y_train, cv=10, n_jobs=-1)
    train_pred[name] = pred_y

    f1 = f1_score(y_train, pred_y)
    acc = accuracy_score(y_train, pred_y)
    results["Name"].append(name)
    results["F1 Score"].append(f1)
    results["Accuracy"].append(acc)
    
    yhat = pd.Series(best_clf.predict(X_test), name="Survived", dtype="int32")
    test_pred[name] = yhat
    output = pd.concat([testIds, yhat], axis=1)
    
    output_filename = name + ".csv"
    print("Writing submission file:", output_filename)
    output.to_csv(f"predictions/{output_filename}", index=False)


Starting training: KNeighborsClassifier
Number of Features: 6
Fitting 10 folds for each of 4 candidates, totalling 40 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.6s finished
Best classifier: KNeighborsClassifier(leaf_size=26, n_neighbors=18)
Writing submission file: KNeighborsClassifier.csv
Starting training: LogisticRegression
Number of Features: 6
Fitting 10 folds for each of 72 candidates, totalling 720 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 628 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best classifier: LogisticRegression(C=0.8, class_weight=1, fit_intercept=False, 

## Hard Voting Classifier

This unites all of the models. It sums the results of the 5 models for each instance and predicts what the majority of models think the output should be.

In [23]:
pred_y = pd.DataFrame.from_dict(train_pred).sum(axis=1) >= 4
f1 = f1_score(y_train, pred_y)
acc = accuracy_score(y_train, pred_y)
results["Name"].append("VotingClassifier")
results["F1 Score"].append(f1)
results["Accuracy"].append(acc)

yhat = pd.Series(pd.DataFrame.from_dict(test_pred).sum(axis=1) >= 4, name="Survived", dtype="int32")
output = pd.concat([testIds, yhat], axis=1)
output_filename = "VotingClassifier.csv"
print("Writing submission file:", output_filename)
output.to_csv(f"predictions/{output_filename}", index=False)

Writing submission file: VotingClassifier.csv


## Results

Now we can take a look at the performance in the train set of each model after cross prediction.

In [24]:
pd.DataFrame.from_dict(results).sort_values(by="Accuracy", ascending=False).reset_index(drop=True)

,Name,F1 Score,Accuracy
0,GradientBoostingClassifier,0.783282,0.842873
1,RandomForestClassifier,0.771930,0.839506
2,KNeighborsClassifier,0.771654,0.837262
3,VotingClassifier,0.764516,0.836139
4,NuSVC,0.754153,0.833895
5,LinearSVC,0.759936,0.830527
6,LinearDiscriminantAnalysis,0.755134,0.826038
7,LogisticRegression,0.758133,0.808081


After submitting the results generated by each model, the NuSVC performed best. It acheived a score of 0.80861, which ranked me among the top 4%.

<img src="TopResult.png"/>